In [1]:
from gensim.models.phrases import Phrases, Phraser
import pandas as pd

import re
import numpy as np
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy
# for plotting
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
# prep NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Read the emails
emails = pd.read_csv("emails.csv")
email_subset = emails.sample(frac=0.02,random_state=1)
print(email_subset.shape)
print(email_subset.head())

(10348, 2)
                              file  \
186822  jones-t/all_documents/634.   
308790  mann-k/all_documents/5690.   
82383         dasovich-j/sent/423.   
227299          kaminski-v/var/63.   
301824     mann-k/_sent_mail/3208.   

                                                  message  
186822  Message-ID: <17820178.1075846925335.JavaMail.e...  
308790  Message-ID: <29110382.1075845717882.JavaMail.e...  
82383   Message-ID: <6812040.1075843194135.JavaMail.ev...  
227299  Message-ID: <21547648.1075856642126.JavaMail.e...  
301824  Message-ID: <12684200.1075846107179.JavaMail.e...  


In [3]:
# Parsing data out from the raw messages

def parse_raw_message(raw_message):
    lines = raw_message.split('\n')
    email = {}
    message = ''
    keys_to_extract = ['message-id', 'from', 'to']
    for line in lines:
        if ':' not in line:
            message += line.strip()
            email['body'] = message
        else:
            pairs = line.split(':')
            key = pairs[0].lower()
            val = pairs[1].strip()
            if key in keys_to_extract:
                email[key] = val
    return email


In [4]:
def map_to_list(emails, key):
    results = []
    for email in emails:
        if key not in email:
            results.append('')
        else:
            results.append(email[key])
    return results

In [5]:
def parse_into_emails(messages):
    emails = [parse_raw_message(message) for message in messages]
    return {
        'body': map_to_list(emails, 'body'),
        'to': map_to_list(emails, 'to'),
        'from_': map_to_list(emails, 'from'),
        'msg_id': map_to_list(emails, 'message-id')
    }


In [6]:
email_df = pd.DataFrame(parse_into_emails(email_subset.message))
print(email_df.head())

                                                body  \
0  It would be nice if you could be at my dinner,...   
1  Absolutely.Good point!  Can Peter start to dra...   
2  My apologies.  My schedule melted down after w...   
3  Vince,UK VAR breached the limit last week.UK t...   
4  Any problems/comments?AM ---------------------...   

                                                  to  \
0                           alicia.goodrow@enron.com   
1                          Kay Mann/Corp/Enron@ENRON   
2                        christine.piesco@oracle.com   
3  Richard Lewis/LON/ECT@ECT, James New/LON/ECT@E...   
4  Don Hammond/PDX/ECT@ECT, Jody Blackburn/PDX/EC...   

                               from_  \
0               tana.jones@enron.com   
1  Sheila Tweed@ECT on 05/15/2001 06   
2            jeff.dasovich@enron.com   
3        tanya.tamarchenko@enron.com   
4                 kay.mann@enron.com   

                                          msg_id  
0  <17820178.1075846925335.JavaMai

In [7]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
print(email_df.iloc[2]['body'])
# Convert email body to list
data = email_df.body.values.tolist()


My apologies.  My schedule melted down after we talked on Monday.  Here'swhere folks came out.  There's some concern about size.  We're supposed to beno larger than 3, but we lobbied Aceves and he apparently Ok'd our"oversized" group.  The other folks in the group--who talked to himoriginally--are pretty sure that five will violate the rules.  Folks wonderedif there were other groups that are smaller than ours that you could hook upwith.  Sorry about that---it's a wrinkle that I didn't think about when wespoke.  If it gets real ugly trying to find a smaller group, let me know.Fortunately there's not another team case due for two weeks.Best,Jeff


In [8]:
# tokenize - break down each sentence into a list of words
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [9]:
data_words = list(sent_to_words(data))

In [10]:
print(data_words[3])


['vince', 'uk', 'var', 'breached', 'the', 'limit', 'last', 'week', 'uk', 'traders', 'asked', 'us', 'to', 'review', 'the', 'correlations', 'across', 'uk', 'gas', 'and', 'power', 'aswell', 'as', 'the', 'correlations', 'across', 'efa', 'slots', 'we', 'did', 'part', 'of', 'the', 'work', 'last', 'week', 'now', 'we', 'll', 'update', 'the', 'correlations', 'based', 'on', 'historical', 'prices', 'tanya', 'richard', 'lewisleppard', 'lon', 'ect', 'ect', 'rudy', 'dautel', 'hou', 'ect', 'ect', 'kirstee', 'hewitt', 'lon', 'ect', 'ect', 'naveen', 'andrews', 'corp', 'enron', 'enron', 'david', 'port', 'market', 'risk', 'corp', 'enron', 'enron', 'tedmurphy', 'hou', 'ect', 'ect', 'simon', 'hastings', 'lon', 'ect', 'ect', 'paul', 'arcy', 'lon', 'ect', 'ect', 'amirghodsian', 'lon', 'ect', 'ectthanks', 'tanya', 'these', 'are', 'interesting', 'results', 'am', 'on', 'vacation', 'next', 'week', 'sohere', 'are', 'my', 'current', 'thoughts', 'am', 'contactable', 'on', 'my', 'mobile', 'if', 'necessary', 'gas', '

In [11]:
# Build the bigram and trigram models
# higher threshold fewer phrases.
bigram = Phrases(data_words, min_count=5, threshold=100)
trigram = Phrases(bigram[data_words], threshold=100)


In [12]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[200]]])


['image_image_image', 'getting', 'together', 'for', 'the', 'holidays', 'is', 'something', 'we', 'all', 'enjoy', 'whether', 'it', 'gathering', 'with', 'old', 'friends', 'from', 'out', 'of', 'town', 'or', 'hanging', 'out', 'with', 'the', 'usual', 'gang', 'every', 'gathering', 'this', 'time', 'of', 'year', 'seems', 'little', 'more', 'special', 'we', 'at', 'miller_brewing', 'wish', 'you', 'many', 'happy', 'celebrations', 'this', 'season', 'and', 'thank', 'you', 'for', 'enjoying', 'those', 'occasions', 'responsibly', 'happy_holidays', 'image', 'this', 'mail', 'is', 'not', 'sent', 'unsolicited', 'you', 'subscribed', 'to', 'receive', 'information', 'from', 'miller_brewing', 'at', 'miller', 'web_site', 'or', 'event', 'must', 'be', 'or', 'older', 'to', 'visit', 'our', 'web_site', 'miller_brewing', 'co', 'milwaukee', 'wi', 'miller_brewing', 'company', 'milwaukee', 'wi', 'privacy_statement', 'image_image_image_image', 'this', 'message', 'was', 'sent', 'by', 'miller_brewing', 'company', 'using', '

In [13]:
# remove stop_words, make bigrams and lemmatize
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append(
            [token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


In [14]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

In [15]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [16]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])


In [17]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [18]:
print(data_lemmatized[200])

['image_image', 'image', 'get', 'together', 'holiday', 'enjoy', 'gather', 'old', 'friend', 'town', 'hang', 'usual', 'gang', 'gathering', 'time', 'year', 'seem', 'little', 'special', 'miller_brewing', 'wish', 'many', 'happy', 'celebration', 'season', 'thank', 'enjoy', 'occasion', 'responsibly', 'happy_holiday', 'image', 'mail', 'send', 'unsolicited', 'subscribe', 'receive', 'information', 'miller_brewe', 'miller', 'web_site', 'event', 'old', 'visit', 'web_site', 'miller_brewe', 'co', 'milwaukee', 'wi', 'miller_brewe', 'company', 'milwaukee', 'wi', 'privacy', 'statement', 'message', 'send', 'miller_brewe', 'company', 'use', 'responsy', 'interact', 'click', 'prefer', 'receive', 'future', 'mail', 'miller_brewe', 'company', 'click', 'view', 'permission', 'marketing', 'policy', 'image']


In [19]:
# create dictionary and corpus both are needed for (LDA) topic modeling

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [21]:
# topic modeling
# corpus, dictionary and number of topics required for LDA
# alpha and eta are hyperparameters that affect sparsity of the topics
# chunksize is the number of documents to be used in each training chunk
# update_every determines how often the model parameters should be updated
# passes is the total number of training passes
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=20,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)


In [22]:
# The weights reflect how important a keyword is to that topic.
print(lda_model.print_topics())


[(0, '0.042*"september" + 0.033*"feature" + 0.025*"license" + 0.012*"strength" + 0.010*"limitation" + 0.010*"unauthorized" + 0.006*"capable" + 0.002*"ifwe" + 0.001*"lp" + 0.001*"dual"'), (1, '0.179*"mail" + 0.061*"message" + 0.031*"et" + 0.028*"email" + 0.019*"table" + 0.018*"org" + 0.018*"address" + 0.017*"font" + 0.017*"delete" + 0.014*"master"'), (2, '0.059*"game" + 0.053*"open" + 0.049*"attempt" + 0.032*"davis" + 0.020*"vote" + 0.019*"measure" + 0.015*"monitor" + 0.015*"senate" + 0.013*"permit" + 0.013*"rest"'), (3, '0.031*"internal" + 0.022*"deny" + 0.019*"sitara" + 0.015*"maintenance" + 0.013*"capability" + 0.012*"pleased" + 0.012*"accurate" + 0.012*"outage" + 0.011*"call_pager" + 0.011*"instance"'), (4, '0.342*"ect" + 0.147*"hou" + 0.121*"enron" + 0.047*"corp" + 0.023*"enronxgate" + 0.013*"lon" + 0.013*"communication" + 0.011*"forward" + 0.010*"ees_ee" + 0.008*"student"'), (5, '0.118*"error" + 0.003*"immediately_notify" + 0.000*"error_occurre" + 0.000*"initialize" + 0.000*"comma

In [23]:
doc_lda = lda_model[corpus]

In [24]:
# Model perplexity and topic coherence provide a convenient
# measure to judge how good a given topic model is.
# Compute Perplexity
# a measure of how good the model is. lower the better.
print('\nPerplexity: ', lda_model.log_perplexity(corpus))



Perplexity:  -15.236878229690644


In [25]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(
    model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Coherence Score:  0.455994351574875


In [26]:
# Visualize the topics
pyLDAvis.enable_notebook(sort=True)
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)


In [27]:
pyLDAvis.display(vis)
